In [ ]:
import os
import cv2
import tqdm
import queue
from thpool import ThreadPool

In [ ]:
def get_filenames(datapath='lane_marking_examples'):
    filenames = []
    for top, dirs, files in os.walk(datapath):
        filenames.extend([os.path.join(top, _file) for _file in files])
    filenames.sort()

    x_paths = [x for x in filenames if not x.endswith('bin.png')]
    y_paths = [x for x in filenames if x.endswith('bin.png')]
    
    return x_paths, y_paths
    
x_paths, y_paths = get_filenames()

In [ ]:
def load_image(filepath, resize=True, pic_h=int(2710 / 4), pic_w = int(3384 / 4)):
    if resize:
        img = cv2.imread(filepath)
    else:
        img = cv2.resize(cv2.imread(y), (pic_h, pic_w), interpolation=cv2.INTER_NEAREST)
        
    return filepath, img

In [ ]:
tasks = queue.Queue()
out = queue.Queue()

for filename in x_paths:
    tasks.put((load_image, (filename, True), {}))

pool = ThreadPool(tasks, out, threads_number=20)
pool.start()
for i in tqdm.tqdm_notebook(range(tasks.qsize())):
    filename, img = out.get()
#     if i > 20:
#         pool.stop()
#         break